In [2]:
# import io
df = pd.read_csv('top_insta_influencers_data.csv')

In [3]:
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3.3k,475.8m,8.7m,1.39%,6.5m,29.0b,Spain
1,2,kyliejenner,91,6.9k,366.2m,8.3m,1.62%,5.9m,57.4b,United States
2,3,leomessi,90,0.89k,357.3m,6.8m,1.24%,4.4m,6.0b,NaN
3,4,selenagomez,93,1.8k,342.7m,6.2m,0.97%,3.3m,11.5b,United States
4,5,therock,91,6.8k,334.1m,1.9m,0.20%,665.3k,12.5b,United States


In [8]:
def convert_to_number(x):
    if 'k' in x:
        return float(x.replace('k', '')) * 1e3
    elif 'm' in x:
        return float(x.replace('m', '')) * 1e6
    elif 'b' in x:
        return float(x.replace('b', '')) * 1e9
    else:
        return float(x)

# Apply conversion to followers and total_likes columns
# df['followers'] = df['followers'].apply(convert_to_number)
# df['total_likes'] = df['total_likes'].apply(convert_to_number)

# Calculate the Engagement Rate (ER) in percentage
df['engagement_rate_%'] = (df['total_likes'] / df['followers']) * 100

# Display the updated DataFrame with calculated ER in percentage
print(df[['channel_info', 'engagement_rate_%']])

    channel_info  engagement_rate_%
0      cristiano        6094.997898
1    kyliejenner       15674.494812
2       leomessi        1679.261125
3    selenagomez        3355.704698
4        therock        3741.394792
..           ...                ...
195    iambeckyg        4216.867470
196   nancyajram        4518.072289
197  luansantana         449.397590
198    nickjonas        5151.515152
199    raisa6690        2954.573171

[200 rows x 2 columns]


In [4]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib

# Custom transformer to preprocess the data
class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def convert_to_number(self, x):
        if isinstance(x, str):
            if 'k' in x:
                return float(x.replace('k', '')) * 1e3
            elif 'm' in x:
                return float(x.replace('m', '')) * 1e6
            elif 'b' in x:
                return float(x.replace('b', '')) * 1e9
        return float(x)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Convert posts to numerical values
        X['total_posts'] = X['posts'].apply(self.convert_to_number)
        
        # Calculate avg_likes_per_post
        X['avg_likes_per_post'] = X['total_likes'] / X['total_posts']
        
        # Calculate engagement_rate_%
        X['engagement_rate_%'] = (X['avg_likes_per_post'] / X['followers']) * 100
        
        # Drop columns that are not used in training
        return X[['rank', 'influence_score', 'followers', 'avg_likes_per_post', 'engagement_rate_%', 'total_posts']]

In [16]:
# Define features and target
X = df[['rank', 'influence_score', 'posts', 'followers', 'total_likes', 'country', 'calculated_er']]
y = df['fake_follower_percentage']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', DataPreprocessor()),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [18]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib

# Custom transformer to preprocess the data
class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def convert_to_number(self, x):
        if isinstance(x, str):
            if 'k' in x:
                return float(x.replace('k', '')) * 1e3
            elif 'm' in x:
                return float(x.replace('m', '')) * 1e6
            elif 'b' in x:
                return float(x.replace('b', '')) * 1e9
        return float(x)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Convert posts to numerical values
        X['total_posts'] = X['posts'].apply(self.convert_to_number)
        
        # Calculate avg_likes_per_post
        X['avg_likes_per_post'] = X['total_likes'] / X['total_posts']
        
        # Calculate engagement_rate_%
        X['engagement_rate_%'] = (X['avg_likes_per_post'] / X['followers']) * 100
        
        # Drop columns that are not used in training
        return X[['rank', 'influence_score', 'followers', 'avg_likes_per_post', 'engagement_rate_%', 'total_posts']]
    
    # Define features and target
X = df[['rank', 'influence_score', 'posts', 'followers', 'total_likes', 'country', 'calculated_er']]
y = df['fake_follower_percentage']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', DataPreprocessor()),
    ('regressor', RandomForestRegressor(random_state=42))
])

    
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model from GridSearchCV
best_pipeline = grid_search.best_estimator_

# Save the best pipeline to a pkl file
joblib.dump(best_pipeline, 'fake_follower_prediction_pipeline.pkl')

print("Pipeline saved to 'fake_follower_prediction_pipeline.pkl'")

Pipeline saved to 'fake_follower_prediction_pipeline.pkl'


In [8]:
import pandas as pd
import joblib

# Load the saved pipeline
pipeline = joblib.load('fake_follower_prediction_pipeline.pkl')

# Example input data (this should be in the same format as used during training)
input_data = pd.DataFrame({
    'rank': [1, 2],
    'influence_score': [92, 91],
    'posts': ['3.3k', '6.9k'],  # This will be converted to numerical values by the pipeline
    'followers': [475800000.0, 366200000.0],
    'total_likes': [2.9e10, 5.74e10],
    'country': ['Spain', 'United States'],
    'calculated_er': [6094.997898, 15674.494812]
})

# Use the pipeline to predict fake follower percentage
predictions = pipeline.predict(input_data)

# Print the predictions
print(predictions)


[7.45881083 0.05613735]


In [5]:
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country,calculated_er
0,1,cristiano,92,3.3k,475800000.0,8.7m,1.39%,6.5m,2.900000e+10,Spain,6094.997898
1,2,kyliejenner,91,6.9k,366200000.0,8.3m,1.62%,5.9m,5.740000e+10,United States,15674.494812
2,3,leomessi,90,0.89k,357300000.0,6.8m,1.24%,4.4m,6.000000e+09,NaN,1679.261125
3,4,selenagomez,93,1.8k,342700000.0,6.2m,0.97%,3.3m,1.150000e+10,United States,3355.704698
4,5,therock,91,6.8k,334100000.0,1.9m,0.20%,665.3k,1.250000e+10,United States,3741.394792


In [9]:
# Apply conversion to posts column to get the total number of posts
df['total_posts'] = df['posts'].apply(convert_to_number)

# Calculate the average likes per post
df['avg_likes_per_post'] = df['total_likes'] / df['total_posts']

# Calculate the Engagement Rate (ER) in percentage
df['engagement_rate_%'] = (df['avg_likes_per_post'] / df['followers']) * 100

# Display the DataFrame with calculated ER in percentage and average likes per post
print(df[['channel_info', 'avg_likes_per_post', 'engagement_rate_%']])

    channel_info  avg_likes_per_post  engagement_rate_%
0      cristiano        8.787879e+06           1.846969
1    kyliejenner        8.318841e+06           2.271666
2       leomessi        6.741573e+06           1.886810
3    selenagomez        6.388889e+06           1.864280
4        therock        1.838235e+06           0.550205
..           ...                 ...                ...
195    iambeckyg        6.086957e+05           1.833421
196   nancyajram        3.947368e+05           1.188966
197  luansantana        1.937662e+05           0.583633
198    nickjonas        7.391304e+05           2.239789
199    raisa6690        2.307381e+05           0.703470

[200 rows x 3 columns]


In [10]:
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country,calculated_er,engagement_rate_%,total_posts,avg_likes_per_post
0,1,cristiano,92,3.3k,475800000.0,8.7m,1.39%,6.5m,2.900000e+10,Spain,6094.997898,1.846969,3300.0,8.787879e+06
1,2,kyliejenner,91,6.9k,366200000.0,8.3m,1.62%,5.9m,5.740000e+10,United States,15674.494812,2.271666,6900.0,8.318841e+06
2,3,leomessi,90,0.89k,357300000.0,6.8m,1.24%,4.4m,6.000000e+09,NaN,1679.261125,1.886810,890.0,6.741573e+06
3,4,selenagomez,93,1.8k,342700000.0,6.2m,0.97%,3.3m,1.150000e+10,United States,3355.704698,1.864280,1800.0,6.388889e+06
4,5,therock,91,6.8k,334100000.0,1.9m,0.20%,665.3k,1.250000e+10,United States,3741.394792,0.550205,6800.0,1.838235e+06


In [11]:
expected_engagement_rate = 2.0  # This could be adjusted based on industry norms

# Calculate the percentage of fake followers
df['fake_follower_percentage'] = (1 - (df['engagement_rate_%'] / expected_engagement_rate)) * 100

# Cap the percentage at 100%
df['fake_follower_percentage'] = df['fake_follower_percentage'].clip(lower=0, upper=100)

# Display the DataFrame with the calculated fake follower percentage
print(df[['channel_info', 'engagement_rate_%', 'fake_follower_percentage']])

    channel_info  engagement_rate_%  fake_follower_percentage
0      cristiano           1.846969                  7.651547
1    kyliejenner           2.271666                  0.000000
2       leomessi           1.886810                  5.659487
3    selenagomez           1.864280                  6.785981
4        therock           0.550205                 72.489744
..           ...                ...                       ...
195    iambeckyg           1.833421                  8.328968
196   nancyajram           1.188966                 40.551680
197  luansantana           0.583633                 70.818338
198    nickjonas           2.239789                  0.000000
199    raisa6690           0.703470                 64.826510

[200 rows x 3 columns]


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [13]:
features = df[['influence_score', 'followers', 'avg_likes_per_post', 'engagement_rate_%', 'total_posts']]
target = df['fake_follower_percentage']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize the RandomForestRegressor
model = RandomForestRegressor(random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model from GridSearch
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Model: {best_model}')
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Best Model: RandomForestRegressor(min_samples_leaf=2, n_estimators=200, random_state=42)
Mean Squared Error: 0.23426249644880626
Mean Absolute Error: 0.3029574854387004
R-squared: 0.9997379962048653


In [12]:
import pandas as pd
import joblib

# Load the saved pipeline
pipeline = joblib.load('fake_follower_prediction_pipeline.pkl')

# Example input data (this should be in the same format as used during training)
input_data = pd.DataFrame({
    'rank': [1, 2, 3],
    'influence_score': [92, 91, 90],
    'posts': ['3.3k', '6.9k', '0.89k'],  # This will be converted to numerical values by the pipeline
    'followers': [475800000.0, 366200000.0, 357300000.0],
    'total_likes': [2.9e10, 5.74e10, 6e9],
    'country': ['Spain', 'United States', 'Argentina'],
    'calculated_er': [6094.997898, 15674.494812, 1679.261125]
})

# Use the pipeline to predict fake follower percentage
predictions = pipeline.predict(input_data)

# Calculate the number of fake followers and real followers
input_data['fake_follower_percentage'] = predictions
input_data['fake_followers'] = (input_data['fake_follower_percentage'] / 100) * input_data['followers']
input_data['real_followers'] = input_data['followers'] - input_data['fake_followers']

# Calculate the total number of influencers, real followers, and fake followers
total_influencers = len(input_data)
total_followers = input_data['followers'].sum()
total_fake_followers = input_data['fake_followers'].sum()
total_real_followers = input_data['real_followers'].sum()

# Calculate percentages
fake_followers_percentage = (total_fake_followers / total_followers) * 100
real_followers_percentage = (total_real_followers / total_followers) * 100

# Print the detailed results
print(input_data[['rank', 'followers', 'fake_follower_percentage', 'fake_followers', 'real_followers']])

# Print the aggregate results
print(f"Total Influencers: {total_influencers}")
print(f"Total Followers: {total_followers}")
print(f"Total Fake Followers: {total_fake_followers} ({fake_followers_percentage:.2f}%)")
print(f"Total Real Followers: {total_real_followers} ({real_followers_percentage:.2f}%)")


   rank    followers  fake_follower_percentage  fake_followers  real_followers
0     1  475800000.0                  7.458811    3.548902e+07    4.403110e+08
1     2  366200000.0                  0.056137    2.055750e+05    3.659944e+08
2     3  357300000.0                  5.869647    2.097225e+07    3.363278e+08
Total Influencers: 3
Total Followers: 1199300000.0
Total Fake Followers: 56666845.66019003 (4.72%)
Total Real Followers: 1142633154.33981 (95.28%)
